In [2]:
# Download and unzip embeddings

! wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
! unzip glove.twitter.27B.zip

In [3]:
import util

embeddings_path = 'glove.twitter.27B.200d.txt'
vocab_path = "./vocab.txt"
SPECIAL_TOKENS = ['<UNK>', '<PAD>', '<SOS>', '<EOS>']

# Download and split data
train_sentences, train_labels, test_sentences, test_labels, label2i = util.load_datasets()
training_sentences, training_labels, dev_sentences, dev_labels = util.split_data(train_sentences, train_labels, split=0.85)

In [4]:
# Set up tokenizer and make vocab
tokenizer = util.Tokenizer()
all_data = train_sentences + test_sentences
tokenized_data = tokenizer.tokenize(all_data)
vocab = sorted(set([w for ws in tokenized_data + [SPECIAL_TOKENS] for w in ws]))
with open('vocab.txt', 'w') as vf:
    vf.write('\n'.join(vocab))

In [5]:
# Load the pretrained embeddings, find the out-of-vocabularies, and add to word2i and embeddings
glove_word2i, glove_embeddings = util.read_pretrained_embeddings(embeddings_path, vocab_path)
oovs = util.get_oovs(vocab_path, glove_word2i)
word2i, embeddings = util.update_embeddings(glove_word2i, glove_embeddings, oovs)

Reading embeddings from glove.twitter.27B.200d.txt...


In [6]:
# Initialize model
import torch

model = util.IronyDetector(
    input_dim=embeddings.shape[1],
    hidden_dim=128,
    embeddings_tensor=embeddings,
    pad_idx=word2i["<PAD>"],
    output_size=len(label2i),
)

In [7]:
# Hyperparameters
batch_size = 8
epochs = 15
learning_rate = 0.00005
weight_decay = 0
optimizer = torch.optim.AdamW(model.parameters(), learning_rate, weight_decay=weight_decay)

# Create batches
batch_tokenized = []
for batch in util.make_batches(training_sentences, batch_size):
    batch_tokenized.append(tokenizer(batch))
batch_labels = util.make_batches(training_labels, batch_size)
dev_sentences = tokenizer(dev_sentences)
test_sentences = tokenizer(test_sentences)

# Encode data
train_features = [util.encode_sentences(batch, word2i) for batch in batch_tokenized]
train_labels = [util.encode_labels(batch) for batch in batch_labels]
dev_features = util.encode_sentences(dev_sentences, word2i)
dev_labels = [int(l) for l in dev_labels]

In [8]:
# Train model
trained_model = util.training_loop(
    epochs,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
    label2i,
)

Training...


  0%|          | 0/408 [00:00<?, ?it/s]

epoch 1, loss: 0.6934011619465024
Evaluating dev...
Dev F1 0.006756756756756757
Avg Dev F1 0.33164940641576157


  0%|          | 0/408 [00:00<?, ?it/s]

epoch 2, loss: 0.6897910628833023
Evaluating dev...
Dev F1 0.5787781350482316
Avg Dev F1 0.5422192562033611


...

epoch 13, loss: 0.537519573580985
Evaluating dev...
Dev F1 0.584493041749503
Avg Dev F1 0.6312295717222092


  0%|          | 0/408 [00:00<?, ?it/s]

epoch 14, loss: 0.5277532096221751
Evaluating dev...
Dev F1 0.5823293172690762
Avg Dev F1 0.6321432519067094


  0%|          | 0/408 [00:00<?, ?it/s]

epoch 15, loss: 0.5180919498351275
Evaluating dev...
Dev F1 0.588469184890656
Avg Dev F1 0.6347584753420923


In [9]:
# Test model
test_features = util.encode_sentences(test_sentences, word2i)
test_labels = [int(l) for l in test_labels]
preds = util.predict(trained_model, test_features)
dev_f1 = util.f1_score(preds, test_labels, label2i['1'])
dev_avg_f1 = util.avg_f1_score(preds, test_labels, set(label2i.values()))
print(f"Test F1 {dev_f1}")
print(f"Avg Test F1 {dev_avg_f1}")

Test F1 0.6035889070146819
Avg Test F1 0.6745693226172886
